In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
IMG_SIZE = 150
categories = ["NORMAL", "PNEUMONIA"]

def create_training_data(DATADIR="../chest_xray/train"):
    training_data = []
    for category in categories:

        path = os.path.join(DATADIR,category)
        class_num = categories.index(category) 
        count = 0
        for img in tqdm(os.listdir(path)):
            count += 1
            if class_num == 1 and count == 1342:
                break
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) 
                if class_num == 0:
                    array = [1, 0]
                else:
                    array = [0, 1]
                training_data.append([new_array, array])
            except Exception as e: 
                pass

    return training_data

training_data = create_training_data("../chest_xray/train")

 35%|███▍      | 1341/3875 [00:05<00:09, 253.61it/s]


In [3]:
test_data = create_training_data("../chest_xray/test")

100%|██████████| 390/390 [00:01<00:00, 292.78it/s]


In [4]:
val_data = create_training_data("../chest_xray/val")

100%|██████████| 8/8 [00:00<00:00, 250.53it/s]


In [5]:
random.shuffle(training_data)
random.shuffle(test_data)
random.shuffle(val_data)

In [6]:
x_train, x_test, x_val = [], [], []
y_train, y_test, y_val = [], [], []


In [7]:
for features,label in training_data:
    x_train.append(features)
    y_train.append(label)
    
for features,label in test_data:
    x_test.append(features)
    y_test.append(label)

for features,label in val_data:
    x_val.append(features)
    y_val.append(label)


In [8]:
np.save("../saved_data/x_train.npy", x_train)
np.save("../saved_data/y_train.npy", y_train)

np.save("../saved_data/x_test.npy", x_test)
np.save("../saved_data/y_test.npy", y_test)

np.save("../saved_data/x_val.npy", x_val)
np.save("../saved_data/y_val.npy", y_val)

x_train = np.array(x_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = np.array(y_train)

x_test = np.array(x_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = np.array(y_test)

x_val = np.array(x_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_val = np.array(y_val)

x_train = x_train/255.
x_test = x_test/255.
x_val = x_val/255.

In [9]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=x_train.shape[1:]))
model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2 , activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val))

Train on 2682 samples, validate on 16 samples
Epoch 1/3
2682/2682 [==============================] - 56s 21ms/sample - loss: 0.6950 - accuracy: 0.5007 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 2/3
2682/2682 [==============================] - 56s 21ms/sample - loss: 0.5769 - accuracy: 0.7144 - val_loss: 0.4551 - val_accuracy: 0.8125
Epoch 3/3
2682/2682 [==============================] - 53s 20ms/sample - loss: 0.2833 - accuracy: 0.9016 - val_loss: 0.2001 - val_accuracy: 0.8750
